In [102]:
from min_metrics import Metrics
import pandas as pd
import re
import numpy as np

In [103]:
global metrics
metrics = Metrics()

In [104]:
def all_scores_(reference_text, hypothesis_text):
    bleu_score = metrics.compute_bleu(reference_text, hypothesis_text)
    # nist_score = metrics.compute_nist(reference_text, hypothesis_text)
    # meteor_s = metrics.compute_meteor(reference_text, hypothesis_text)
    # rouge_scores = metrics.compute_rouge(reference_text, hypothesis_text)
    # em = metrics.exact_match(reference_text, hypothesis_text)
    # levenshtein_dist = metrics.levenshtein_distance(reference_text, hypothesis_text)
    damerau_levenshtein_score = metrics.damerau_levenshtein(reference_text, hypothesis_text)
    # word_error_rate_score = metrics.word_error_rate(reference_text, hypothesis_text)
    # gestalt_score = metrics.gestalt_pattern_matching(reference_text, hypothesis_text)
    # jaccard_score = metrics.jaccard(reference_text, hypothesis_text)
    # overlap_coeff_score = metrics.overlap_coeff(reference_text, hypothesis_text)
    # dice_sorensen_score = metrics.dice_sorensen_coefficient(reference_text, hypothesis_text)
    tf_idf_score = metrics.compute_cosine_similarity(reference_text, hypothesis_text, "tf-idf")
    # word_freq_score = metrics.compute_cosine_similarity(reference_text, hypothesis_text, "word_frequency")
    # spacy_similarity = metrics.semantic_similarity_spacy(reference_text, hypothesis_text)
    # wmd_score = metrics.wmd(reference_text, hypothesis_text)
    # gensim_cosine_score = metrics.gensim_cosine(reference_text, hypothesis_text)
    # sbert_score = metrics.sbert_cosine(reference_text, hypothesis_text)
    use_score = metrics.USE_similarity(reference_text, hypothesis_text)
    # soft_cosine_score = metrics.soft_cosine(reference_text, hypothesis_text)
    return [bleu_score, damerau_levenshtein_score, tf_idf_score, use_score]
    return [
        bleu_score, nist_score, meteor_s, rouge_scores, em, levenshtein_dist, 
        damerau_levenshtein_score, word_error_rate_score, gestalt_score, jaccard_score, 
        overlap_coeff_score, dice_sorensen_score, tf_idf_score, word_freq_score, 
        spacy_similarity, wmd_score, gensim_cosine_score, sbert_score, use_score, soft_cosine_score
    ]


In [105]:
reference_text = "The transition from winter to spring is one of the most anticipated times of the year. The cold weather begins to fade, and the warmth of the sun gradually returns. As the days grow longer, nature awakens, with flowers blooming and trees regrowing their leaves. People feel renewed, eager to spend more time outdoors and enjoy the fresh air. It is a season of hope, growth, and change."
hypothesis_text = "The transition from winter to spring is one of the most anticipated times of the year. The cold begins to disappear and the heat of the sun gradually returns. As the days get longer, nature awakens, flowers bloom and trees grow again. People feel refreshed and look forward to spending more time outdoors enjoying the fresh air. It is a time of hope, growth and change."
all_scores_(reference_text, hypothesis_text)

[0.8260465164578865,
 0.7844155844155845,
 np.float64(0.8603579934292734),
 np.float32(0.9504762)]

In [142]:
files = {
    "gpt": "../data/chatgpt_llm_answers_new_prompt.csv",
    "claude": "../data/claude_llm_answers_new_prompt.csv",
    "deepseek": "../data/deepseek_llm_answers_new_prompt.csv",
    "gemini": "../data/gemini_llm_answers_new_prompt.csv"
}
column_key_map = {
    "Expert Key Factors in Consideration": "Key Factors",
    "Expert Proposed Resolution Strategies": "Proposed Resolution Strategies",
    "Expert Key Takeaways": "Key Takeaways"
}

model_dfs = {model: pd.read_csv(path) for model, path in files.items()}

base_df = model_dfs["claude"].copy()
non_expert_columns = [col for col in base_df.columns if col not in column_key_map]

def build_expert_dict(row_idx):
    combined_dict = {}
    for model, df in model_dfs.items():
        combined_dict[model] = {
            column_key_map[col]: df.loc[row_idx, col] for col in column_key_map
        }
    return combined_dict

base_df["Expert Data Dictionary"] = base_df.index.map(build_expert_dict)

final_df = base_df[non_expert_columns + ["Expert Data Dictionary"]]
final_df = final_df.drop(["Unnamed: 0", "dilemma_output", "Dilemma Key Factors in Consideration", "Dilemma Proposed Resolution Strategies", "Dilemma Key Takeaways"],axis = 1)
#final_df = final_df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
final_df.head(5)

,Dilemma Description,Expert Opinion,expert_output,Expert Data Dictionary
0,I joined a lab during graduate school and was ...,We were surprised to discover that the literat...,%Key Factors in Consideration:\n1. Absence of ...,{'gpt': {'Key Factors': '1. Lack of establishe...
1,Graduate students A and B are working on somew...,Disputes over assigning credit in science are ...,%Key Factors in Consideration:\n1. Competitive...,{'gpt': {'Key Factors': '1. Intense competitio...
2,David is a new postdoc in Dr. Goliath’s lab. U...,A key source of ethical guidance in resolving ...,%Key Factors in Consideration:\n1. ICMJE Autho...,{'gpt': {'Key Factors': '1. ICMJE authorship c...
3,"A PI moves his lab to a different university, ...","At first blush, this PI certainly seems to be ...",%Key Factors in Consideration:\n1. Power dynam...,{'gpt': {'Key Factors': '1. Ethical implicatio...
4,George Washington is one of two postdocs worki...,"Dr. Big might be a fine scientist, but he’s no...",%Key Factors in Consideration:\n1. Institution...,{'gpt': {'Key Factors': '1. Misrepresentation ...


In [143]:
final_df["Expert Data Dictionary"][0]["claude"].keys()

dict_keys(['Key Factors', 'Proposed Resolution Strategies', 'Key Takeaways'])

In [144]:
final_df.shape

(55, 4)

In [145]:
final_df["Expert Data Dictionary"][0]["claude"]["Key Takeaways"]

'1. Scientific credit allocation is inherently complex and requires nuanced, context-specific evaluation.\n\n2. Power imbalances and institutional hierarchies can significantly impact how scientific contributions are recognized and valued.\n\n3. Collaborative research requires careful consideration of individual contributions beyond simple hierarchical assumptions.\n\n4. Transparent and impartial review processes are crucial for fairly resolving disputes about scientific credit.\n\n5. Graduate students must be proactive in protecting their intellectual contributions and choosing supportive research environments.'

In [156]:
gemini = pd.read_csv("../data/llama3.2-1b_llm_answers_new_prompt.csv")
gemini = gemini[["Expert Opinion", "dilemma_output", "Dilemma Key Factors in Consideration", "Dilemma Proposed Resolution Strategies", "Dilemma Key Takeaways"]]
gemini.head(5)

,Expert Opinion,dilemma_output,Dilemma Key Factors in Consideration,Dilemma Proposed Resolution Strategies,Dilemma Key Takeaways
0,We were surprised to discover that the literat...,Here is the analysis of the ethical dilemma in...,1. The post-doc's claim to have verified the c...,1. Documenting the timeline: Establish a clear...,1. Clear documentation is essential for establ...
1,Disputes over assigning credit in science are ...,Key Factors in Consideration:\n\n1. Intellectu...,NaN,NaN,NaN
2,A key source of ethical guidance in resolving ...,%Key Factors in Consideration:\n\n1. The orig...,1. The original author's contribution was sub...,1. David should have approached Dr. Goliath a...,1. The replacement of an original author with...
3,"At first blush, this PI certainly seems to be ...",%Key Factors in Consideration:\n\n1. Instituti...,1. Institutional affiliation and privilege: Th...,1. Clear communication about new policy implic...,1. The PI's decision raises concerns about ins...
4,"Dr. Big might be a fine scientist, but he’s no...",Key Factors in Consideration:\n\n1. Dr. Big's...,NaN,NaN,NaN


In [157]:
gemini["Expert Opinion"][0] == final_df["Expert Opinion"][0]

True

In [158]:
merged = final_df.merge(gemini, how='left', on='Expert Opinion')
merged.head(5)

,Dilemma Description,Expert Opinion,expert_output,Expert Data Dictionary,dilemma_output,Dilemma Key Factors in Consideration,Dilemma Proposed Resolution Strategies,Dilemma Key Takeaways
0,I joined a lab during graduate school and was ...,We were surprised to discover that the literat...,%Key Factors in Consideration:\n1. Absence of ...,{'gpt': {'Key Factors': '1. Lack of establishe...,Here is the analysis of the ethical dilemma in...,1. The post-doc's claim to have verified the c...,1. Documenting the timeline: Establish a clear...,1. Clear documentation is essential for establ...
1,Graduate students A and B are working on somew...,Disputes over assigning credit in science are ...,%Key Factors in Consideration:\n1. Competitive...,{'gpt': {'Key Factors': '1. Intense competitio...,Key Factors in Consideration:\n\n1. Intellectu...,NaN,NaN,NaN
2,David is a new postdoc in Dr. Goliath’s lab. U...,A key source of ethical guidance in resolving ...,%Key Factors in Consideration:\n1. ICMJE Autho...,{'gpt': {'Key Factors': '1. ICMJE authorship c...,%Key Factors in Consideration:\n\n1. The orig...,1. The original author's contribution was sub...,1. David should have approached Dr. Goliath a...,1. The replacement of an original author with...
3,"A PI moves his lab to a different university, ...","At first blush, this PI certainly seems to be ...",%Key Factors in Consideration:\n1. Power dynam...,{'gpt': {'Key Factors': '1. Ethical implicatio...,%Key Factors in Consideration:\n\n1. Instituti...,1. Institutional affiliation and privilege: Th...,1. Clear communication about new policy implic...,1. The PI's decision raises concerns about ins...
4,George Washington is one of two postdocs worki...,"Dr. Big might be a fine scientist, but he’s no...",%Key Factors in Consideration:\n1. Institution...,{'gpt': {'Key Factors': '1. Misrepresentation ...,Key Factors in Consideration:\n\n1. Dr. Big's...,NaN,NaN,NaN


In [159]:
#compute_cosine_similarity
#damerau_levenshtein
#semantic_similarity_spacy
#USE_similarity
#compute_bleu

In [ ]:
def split_numbered_list(text):
    if pd.isna(text):
        return []
    
    parts = re.split(r'\d+\.\s*', text)
    return [p.strip() for p in parts if p.strip()]

def calculate_score(df, metric, metric_name):
    cols = {
        "Key Factors": "Dilemma Key Factors in Consideration",
        "Proposed Resolution Strategies": "Dilemma Proposed Resolution Strategies",
        "Key Takeaways": "Dilemma Key Takeaways"
    }

    models = ["gpt", "claude", "deepseek", "gemini"]

    for section_key, dil_col in cols.items():

        def average_model_score(row):
            dil_text = row[dil_col]
            expert_dict = row["Expert Data Dictionary"]
            model_score = []

            model_bullets = split_numbered_list(dil_text)

            for m in models:
                ground_truth_text = expert_dict[m][section_key] if not pd.isna(expert_dict[m][section_key]) else ""
                ground_truth_bullets = split_numbered_list(ground_truth_text)

                bullet_scores = []

                for gt_bullet in ground_truth_bullets:
                    if model_bullets:
                        max_score = max([metric(gt_bullet, hyp_bullet) for hyp_bullet in model_bullets])
                    else:
                        max_score = 0
                    bullet_scores.append(max_score)

                for hyp_bullet in model_bullets:
                    if ground_truth_bullets:
                        max_score = max([metric(hyp_bullet, gt_bullet) for gt_bullet in ground_truth_bullets])
                    else:
                        max_score = 0
                    bullet_scores.append(max_score)
                    

                if bullet_scores:
                    model_score.append(sum(bullet_scores) / len(bullet_scores))
                else:
                    model_score.append(0)

            return sum(model_score) / len(model_score)

        df[f"Score - {section_key} - {metric_name}"] = df.apply(average_model_score, axis=1)
    
    df[f"Total Score {metric_name}"] = (
        df[f"Score - Key Factors - {metric_name}"] * (0.2 / 0.75) +
        df[f"Score - Proposed Resolution Strategies - {metric_name}"] * (0.33 / 0.75) +
        df[f"Score - Key Takeaways - {metric_name}"] * (0.22 / 0.75)
    )
    return df


In [161]:
met = [metrics.compute_cosine_similarity, metrics.damerau_levenshtein, metrics.USE_similarity, metrics.compute_bleu]
met_names = ["Tf-idf", "DL", "USE", "BLEU"]

for i in range (len(met)):
    score_table = calculate_score(merged, met[i], met_names[i])

score_table.head(5)

,Dilemma Description,Expert Opinion,expert_output,Expert Data Dictionary,dilemma_output,Dilemma Key Factors in Consideration,Dilemma Proposed Resolution Strategies,Dilemma Key Takeaways,Score - Key Factors - Tf-idf,Score - Proposed Resolution Strategies - Tf-idf,...,Score - Key Takeaways - DL,Total Score DL,Score - Key Factors - USE,Score - Proposed Resolution Strategies - USE,Score - Key Takeaways - USE,Total Score USE,Score - Key Factors - BLEU,Score - Proposed Resolution Strategies - BLEU,Score - Key Takeaways - BLEU,Total Score BLEU
0,I joined a lab during graduate school and was ...,We were surprised to discover that the literat...,%Key Factors in Consideration:\n1. Absence of ...,{'gpt': {'Key Factors': '1. Lack of establishe...,Here is the analysis of the ethical dilemma in...,1. The post-doc's claim to have verified the c...,1. Documenting the timeline: Establish a clear...,1. Clear documentation is essential for establ...,0.288952,0.160563,...,0.295235,0.292163,0.415136,0.344646,0.336045,0.360920,0.303762,0.302711,0.326277,0.309904
1,Graduate students A and B are working on somew...,Disputes over assigning credit in science are ...,%Key Factors in Consideration:\n1. Competitive...,{'gpt': {'Key Factors': '1. Intense competitio...,Key Factors in Consideration:\n\n1. Intellectu...,NaN,NaN,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,David is a new postdoc in Dr. Goliath’s lab. U...,A key source of ethical guidance in resolving ...,%Key Factors in Consideration:\n1. ICMJE Autho...,{'gpt': {'Key Factors': '1. ICMJE authorship c...,%Key Factors in Consideration:\n\n1. The orig...,1. The original author's contribution was sub...,1. David should have approached Dr. Goliath a...,1. The replacement of an original author with...,0.299177,0.271014,...,0.288025,0.288951,0.466536,0.433181,0.420167,0.438258,0.259435,0.323366,0.390642,0.326052
3,"A PI moves his lab to a different university, ...","At first blush, this PI certainly seems to be ...",%Key Factors in Consideration:\n1. Power dynam...,{'gpt': {'Key Factors': '1. Ethical implicatio...,%Key Factors in Consideration:\n\n1. Instituti...,1. Institutional affiliation and privilege: Th...,1. Clear communication about new policy implic...,1. The PI's decision raises concerns about ins...,0.225381,0.217431,...,0.284677,0.283540,0.416747,0.413609,0.411364,0.413787,0.352339,0.358812,0.372150,0.360998
4,George Washington is one of two postdocs worki...,"Dr. Big might be a fine scientist, but he’s no...",%Key Factors in Consideration:\n1. Institution...,{'gpt': {'Key Factors': '1. Misrepresentation ...,Key Factors in Consideration:\n\n1. Dr. Big's...,NaN,NaN,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [162]:
score_table.columns

Index(['Dilemma Description', 'Expert Opinion', 'expert_output',
       'Expert Data Dictionary', 'dilemma_output',
       'Dilemma Key Factors in Consideration',
       'Dilemma Proposed Resolution Strategies', 'Dilemma Key Takeaways',
       'Score - Key Factors - Tf-idf',
       'Score - Proposed Resolution Strategies - Tf-idf',
       'Score - Key Takeaways - Tf-idf', 'Total Score Tf-idf',
       'Score - Key Factors - DL',
       'Score - Proposed Resolution Strategies - DL',
       'Score - Key Takeaways - DL', 'Total Score DL',
       'Score - Key Factors - USE',
       'Score - Proposed Resolution Strategies - USE',
       'Score - Key Takeaways - USE', 'Total Score USE',
       'Score - Key Factors - BLEU',
       'Score - Proposed Resolution Strategies - BLEU',
       'Score - Key Takeaways - BLEU', 'Total Score BLEU'],
      dtype='object')

In [163]:
score_table["Final Score"] = score_table["Total Score DL"] * 0.0768  + score_table["Total Score BLEU"] * 0.1547 + score_table["Total Score USE"] * 0.5386 + score_table["Total Score Tf-idf"] * 0.2299
score_table.head(5)

,Dilemma Description,Expert Opinion,expert_output,Expert Data Dictionary,dilemma_output,Dilemma Key Factors in Consideration,Dilemma Proposed Resolution Strategies,Dilemma Key Takeaways,Score - Key Factors - Tf-idf,Score - Proposed Resolution Strategies - Tf-idf,...,Total Score DL,Score - Key Factors - USE,Score - Proposed Resolution Strategies - USE,Score - Key Takeaways - USE,Total Score USE,Score - Key Factors - BLEU,Score - Proposed Resolution Strategies - BLEU,Score - Key Takeaways - BLEU,Total Score BLEU,Final Score
0,I joined a lab during graduate school and was ...,We were surprised to discover that the literat...,%Key Factors in Consideration:\n1. Absence of ...,{'gpt': {'Key Factors': '1. Lack of establishe...,Here is the analysis of the ethical dilemma in...,1. The post-doc's claim to have verified the c...,1. Documenting the timeline: Establish a clear...,1. Clear documentation is essential for establ...,0.288952,0.160563,...,0.292163,0.415136,0.344646,0.336045,0.360920,0.303762,0.302711,0.326277,0.309904,0.309142
1,Graduate students A and B are working on somew...,Disputes over assigning credit in science are ...,%Key Factors in Consideration:\n1. Competitive...,{'gpt': {'Key Factors': '1. Intense competitio...,Key Factors in Consideration:\n\n1. Intellectu...,NaN,NaN,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,David is a new postdoc in Dr. Goliath’s lab. U...,A key source of ethical guidance in resolving ...,%Key Factors in Consideration:\n1. ICMJE Autho...,{'gpt': {'Key Factors': '1. ICMJE authorship c...,%Key Factors in Consideration:\n\n1. The orig...,1. The original author's contribution was sub...,1. David should have approached Dr. Goliath a...,1. The replacement of an original author with...,0.299177,0.271014,...,0.288951,0.466536,0.433181,0.420167,0.438258,0.259435,0.323366,0.390642,0.326052,0.366894
3,"A PI moves his lab to a different university, ...","At first blush, this PI certainly seems to be ...",%Key Factors in Consideration:\n1. Power dynam...,{'gpt': {'Key Factors': '1. Ethical implicatio...,%Key Factors in Consideration:\n\n1. Instituti...,1. Institutional affiliation and privilege: Th...,1. Clear communication about new policy implic...,1. The PI's decision raises concerns about ins...,0.225381,0.217431,...,0.283540,0.416747,0.413609,0.411364,0.413787,0.352339,0.358812,0.372150,0.360998,0.348937
4,George Washington is one of two postdocs worki...,"Dr. Big might be a fine scientist, but he’s no...",%Key Factors in Consideration:\n1. Institution...,{'gpt': {'Key Factors': '1. Misrepresentation ...,Key Factors in Consideration:\n\n1. Dr. Big's...,NaN,NaN,NaN,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [164]:
np.mean(score_table["Final Score"])

np.float64(0.09118220626529987)

In [ ]:
score_table.to_csv("llama3.2-3b_new_metrics.csv")